1\. Write a function that converts number representation (bin<->dec<->hex)

In [51]:
#only for positive integers



def bintodec(input):
    pos= [int(x) for x in input[2:]]
    pos= pos[::-1] #reversing the order of the array
    temp=0 
    for i in range(len(pos)): temp+=2**i * pos[i]
    return temp





def converter(x, intype):
    out=[]
    if intype==int:
    # conversion to binary:
        out1=bintodec(x)


    if intype== str and ('b' in intype) == True:
        #conversion to decimal:
        
    if intype== str and ('x' in intype) == True:





IndentationError: expected an indented block (<ipython-input-51-9a5f99103c47>, line 16)

In [133]:
x=[0,1,1]

s

'011'

In [123]:
def DecimalToBinary(num):
    temp=[]
    if num >= 1: DecimalToBinary(num // 2)
    temp.append(num % 2)
    

def DecimalToBinary(num):
    temp=[]
    while (num // 2 >=1):
        temp.append(num%2)
        num=num//2
    #temp.append)
    if num != 0: temp.append(1)
    temp=temp[::-1]
    s=''
    for i in range(len(temp)):s+=str(x[i])
    return(s)
DecimalToBinary(4)

[1, 0, 0]

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
